## support vector machine con grid search

In [2]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import LeaveOneOut
from sklearn import metrics
import pandas as pd
from matplotlib import pyplot as plt
from statistics import mean
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


#setting up labels for dataset
labels = ['class', 'spec_number', 'eccentr', 'asp_ratio', 'elong', 'solidity', 'stoch_conv', 'iso_factor', 'max_ind_depth', 'lobedness', 'av_intensity', 'av_contr', 'smooth', 'third_mom', 'unif', 'entropy']
#importing data
df = pd.read_csv(r'./leaf/leaf.csv', header = None, names = labels)
# shuffling the dataframe
df = df.sample(frac=1).reset_index()
df = df.iloc[:, 1:17]   # needed to eliminate the old indexes column
#display(df)
#separating y from x
X = df.iloc[:, 1:16]
y = df.iloc[:, 0]

display(X)

,spec_number,eccentr,asp_ratio,elong,solidity,stoch_conv,iso_factor,max_ind_depth,lobedness,av_intensity,av_contr,smooth,third_mom,unif,entropy
0,12,0.45767,1.1629,0.24078,0.94275,0.99298,0.63919,0.020279,0.074844,0.028432,0.095746,0.009084,0.002812,0.000148,0.78535
1,1,0.99571,10.3520,0.90609,0.90652,0.97895,0.16732,0.025787,0.121030,0.023523,0.089455,0.007939,0.002888,0.000069,0.73774
2,3,0.24580,1.1011,0.20617,0.94995,0.99298,0.53189,0.012818,0.029901,0.142790,0.205370,0.040470,0.009027,0.001809,2.57900
3,5,0.99799,15.0680,0.93667,0.88070,0.95789,0.12132,0.032164,0.188280,0.011686,0.054827,0.002997,0.000864,0.000044,0.50576
4,8,0.46590,1.2195,0.27095,0.94433,1.00000,0.66599,0.013322,0.032299,0.024332,0.104050,0.010710,0.004713,0.000046,0.56447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,1,0.73365,1.5028,0.34887,0.98280,1.00000,0.80144,0.005365,0.005238,0.029255,0.103270,0.010553,0.004301,0.000092,0.88407
336,8,0.49634,1.1832,0.22855,0.94687,0.98947,0.58613,0.017327,0.054638,0.047106,0.127480,0.015992,0.005173,0.000239,1.14890
337,2,0.90391,2.4580,0.59770,0.98610,1.00000,0.61429,0.006701,0.008173,0.089020,0.167060,0.027151,0.007219,0.000966,1.89370
338,2,0.90557,2.3423,0.58487,0.95943,0.97368,0.55537,0.023542,0.100870,0.045897,0.134330,0.017724,0.006676,0.000245,0.99430


#### grid search con k-fold CV e LOOCV

domande:

- cos'è rbf (in kernel)?

- il rescaling delle variabili come si fa (probabilmente lo fa automaticamente)?

- C è la tolleranza (in particolare corrisponde alla prima o alla seconda formulazione di Medvet)?

- perché roc_auc_ovo e roc_auc_ovr non funzionano?

- cercare modo migliore per 'elencare' i regularization parameter

domande generali:

- ha senso tenere spec_number nel dataset?

- come si fa a mettere più di un effectivness index nella grid search?

In [17]:
k = 5

# building the range of the regularization parameter (C)
exp = np.arange(-8, 8)
reg_param = 10.**exp

grid_param = {'C': reg_param,
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
              'degree': np.arange(2, 5),
              'decision_function_shape': ['ovo', 'ovr']}

clf_cv = GridSearchCV(svm.SVC(), grid_param, cv=k, scoring='accuracy')
clf_cv.fit(X, y)
print(clf_cv.best_params_)
print(clf_cv.best_score_)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': array([1.e-08, 1.e-07, 1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01,
       1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06, 1.e+07]),
                         'decision_function_shape': ['ovo', 'ovr'],
                         'degree': array([2, 3, 4]),
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='accuracy')

{'C': 100000.0, 'decision_function_shape': 'ovo', 'degree': 2, 'kernel': 'linear'}
0.7058823529411764


In [3]:
# building the range of the regularization parameter (C)
exp = np.arange(-8, 8)
reg_param = 10.**exp

grid_param = {'C': reg_param,
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
              'degree': np.arange(2, 5),
              'decision_function_shape': ['ovo', 'ovr']}

clf_loocv = GridSearchCV(svm.SVC(), grid_param, cv=LeaveOneOut(), scoring='accuracy')
clf_loocv.fit(X, y)
print(clf_loocv.best_params_)
print(clf_loocv.best_score_)

KeyboardInterrupt: 